# Chage the annotations format to be compatible with YOLOv11-OBB

In [ ]:
pip install pickleshare

In [ ]:
cd /workspace/samgtgenerate2025

In [ ]:
pip install -r requirements.txt

### Use segment2YOLOGT to create masks of the annotations using SAM

In [ ]:
!python segment2YOLOGT.py -i /workspace/C_Split_80_10_10/training/images -x /workspace/C_Split_80_10_10/training/Annotations -o C_80_10_10_train --ckp sam_vit_h_4b8939.pth --ln --classes /workspace/samgtgenerate2025/XML2Img-classes.json

### Transform polygons generated with SAM to OBB annotations

YOLOv11-OBB uses annotations with this structure: class_index x1 y1 x2 y2 x3 y3 x4 y4

In [ ]:
#CODE THAT TRANSFORMS POLYGONS GENERATED WITH SAM TO OBB ANNOTATIONS
import os
import cv2
import numpy as np

def polygon_to_obb(line, img_width, img_height):
    # Split input line into class id + polygon coordinates
    data = line.strip().split()
    if len(data) < 10:  # must have class id + at least 4 points
        return None  
    
    try:
        class_id = int(data[0])
        coords = list(map(float, data[1:]))
        if len(coords) % 2 != 0:  # coordinates must be in pairs
            return None  

        # Convert to (N,2) array and scale from normalized [0,1] → pixels
        coords = np.array(coords, dtype=np.float32).reshape(-1, 2)
        coords[:, 0] *= img_width
        coords[:, 1] *= img_height

        # Fit minimum-area rectangle around polygon
        cnt = coords.reshape(-1, 1, 2)
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)

        # Clip to image bounds and normalize back to [0,1]
        box = np.clip(box, [0, 0], [img_width, img_height])
        box[:, 0] /= img_width
        box[:, 1] /= img_height

        # Return as YOLO-style string: class_id + 8 coords
        flattened = box.flatten()
        return f"{class_id} " + " ".join(f"{x:.6f}" for x in flattened)
    except Exception as e:
        return None  


def convert_folder(input_dir, output_dir, img_width, img_height):
    # Create output directory if it doesn’t exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all annotation files
    for filename in sorted(os.listdir(input_dir)):
        if not filename.endswith('.txt'):
            continue

        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Read input annotation and write converted OBB annotations
        with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
            count = 0
            for line in infile:
                count += 1
                obb_line = polygon_to_obb(line, img_width, img_height)
                if obb_line:
                    outfile.write(obb_line + '\n')

            # Log how many lines were processed in the file
            print(f"{filename}: {count} vehicles processed.")

# MAIN
input_dir = '/workspace/C_polygons_labels/labels/' 
output_dir = '/workspace/C_polygons_labels/OBB/'      
img_width = 3.840                 
img_height = 2.160                    
convert_folder(input_dir, output_dir, img_width, img_height)
